In [1]:
#! /bin/env python
# -*- coding: utf-8 -*-
"""
预测
"""
import jieba
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
from snownlp import SnowNLP as mynlptool
import yaml
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)

# define parameters
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')


def input_transform(string):
    words=jieba.lcut(string)
    words=np.array(words).reshape(1,-1)
    model=Word2Vec.load('Word2vec_model.pkl')
    _,_,combined=create_dictionaries(model,words)
    return combined


def lstm_predict(string):
    print ('loading model......')
    with open('lstm.yml', 'r') as f:
        yaml_string = yaml.load(f)
    model = model_from_yaml(yaml_string)

    print ('loading weights......')
    model.load_weights('lstm.h5')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    data=input_transform(string)
    data.reshape(1,-1)
    #print data
    result=model.predict_classes(data)
    # print result # [[1]]
    if result[0]==1:
        print (string,' positive')
    elif result[0]==0:
        print (string,' neural')
    else:
        print (string,' negative')


# string='酒店的环境非常好，价格也便宜，值得推荐'
# string='手机质量太差了，傻逼店家，赚黑心钱，以后再也不会买了'
# string = "这是我看过文字写得很糟糕的书，因为买了，还是耐着性子看完了，但是总体来说不好，文字、内容、结构都不好"
# string = "虽说是职场指导书，但是写的有点干涩，我读一半就看不下去了！"
# string = "书的质量还好，但是内容实在没意思。本以为会侧重心理方面的分析，但实际上是婚外恋内容。"
# string = "不是太好"
# string = "不错不错"
# string = "真的一般，没什么可以学习的"

# lstm_predict(string)

d:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
# string='酒店的环境非常好，价格也便宜，值得推荐'
# string='手机质量太差了，傻逼店家，赚黑心钱，以后再也不会买了'
# string = "这是我看过文字写得很糟糕的书，因为买了，还是耐着性子看完了，但是总体来说不好，文字、内容、结构都不好"
# string = "虽说是职场指导书，但是写的有点干涩，我读一半就看不下去了！"
# string = "书的质量还好，但是内容实在没意思。本以为会侧重心理方面的分析，但实际上是婚外恋内容。"
# string = "不是太好"
# string = "不错不错"
# string = "真的一般，没什么可以学习的"
string = '这个股票下跌，糟糕透了'

lstm_predict(string)

loading model......
loading weights......


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


这个股票下跌，糟糕透了  neural


进行情感分析

In [2]:


# string='酒店的环境非常好，价格也便宜，值得推荐'
string='手机质量太差了，傻逼店家，赚黑心钱，以后再也不会买了'
# string = "这是我看过文字写得很糟糕的书，因为买了，还是耐着性子看完了，但是总体来说不好，文字、内容、结构都不好"
# string = "虽说是职场指导书，但是写的有点干涩，我读一半就看不下去了！"
# string = "书的质量还好，但是内容实在没意思。本以为会侧重心理方面的分析，但实际上是婚外恋内容。"
# string = "不是太好"





In [7]:
string='这家公司太差了，傻逼公司，赚黑心钱，以后再也不会买了'
s = mynlptool(string)
s.sentiments, str(s.words), str(list(s.tags))

(0.0004931080011100208,
 "['这家', '公司', '太差', '了', '，', '傻', '逼', '公司', '，', '赚', '黑心', '钱', '，', '以后', '再', '也', '不', '会', '买', '了']",
 "[('这家', 'r'), ('公司', 'n'), ('太差', 'Ag'), ('了', 'y'), ('，', 'w'), ('傻', 'a'), ('逼', 'v'), ('公司', 'n'), ('，', 'w'), ('赚', 'v'), ('黑心', 'nr'), ('钱', 'nr'), ('，', 'w'), ('以后', 'f'), ('再', 'd'), ('也', 'd'), ('不', 'd'), ('会', 'v'), ('买', 'v'), ('了', 'y')]")

In [6]:
string='这只股票非常好，价格也便宜，值得推荐'
s = mynlptool(string)
s.sentiments, str(s.words), str(list(s.tags))

(0.971132196567387,
 "['这', '只', '股票', '非常', '好', '，', '价格', '也', '便宜', '，', '值得', '推荐']",
 "[('这', 'r'), ('只', 'q'), ('股票', 'n'), ('非常', 'd'), ('好', 'a'), ('，', 'w'), ('价格', 'n'), ('也', 'd'), ('便宜', 'a'), ('，', 'w'), ('值得', 'v'), ('推荐', 'v')]")

In [3]:
string = "股票快涨"
s = mynlptool(string)
s.sentiments, str(s.words), str(list(s.tags))

(0.9331816845145019,
 "['股票', '快', '涨']",
 "[('股票', 'n'), ('快', 'd'), ('涨', 'v')]")

In [5]:
string = "特别垃圾的股票"
s = mynlptool(string)
s.sentiments, str(s.words), str(list(s.tags))

(0.35270721476344424,
 "['特别', '垃圾', '的', '股票']",
 "[('特别', 'a'), ('垃圾', 'n'), ('的', 'u'), ('股票', 'n')]")

In [2]:
import pandas as pd

In [3]:
combine_df = pd.read_csv('combine_df.csv')

In [78]:
combine_df.head()

,发表时间,评论标题,sentiment_value,sentiment_label
0,20160401,等待非农就业数据 美股午盘维持小幅上扬,0.897162,1
1,20160401,4月1日股市早盘内参：上交所决定4月8日起暂停*S,0.250485,0
2,20160401,净买入再超20亿元 沪股通扫货四大券商股,0.304655,0
3,20160401,专家:三月反攻圆满收官 四月行情值得期待,0.991793,1
4,20160401,梧桐树扎根A股“一箭双雕”,0.992092,1


In [9]:
sentiment_list = []

In [10]:
for i in combine_df['评论标题']:
    s = mynlptool(i)
    sentiment_list.append(s.sentiments)

In [13]:
len(sentiment_list), combine_df.shape

(112380, (112380, 2))

In [14]:
combine_df['sentiment_value'] = sentiment_list

In [96]:
combine_df.head(50)

,date,news_title,sentiment_value,sentiment_label
0,2016-04-01 00:03:56,等待非农就业数据 美股午盘维持小幅上扬,0.897162,1
1,2016-04-01 01:01:06,4月1日股市早盘内参：上交所决定4月8日起暂停*S,0.250485,0
2,2016-04-01 01:19:11,净买入再超20亿元 沪股通扫货四大券商股,0.304655,0
3,2016-04-01 01:22:06,专家:三月反攻圆满收官 四月行情值得期待,0.991793,1
4,2016-04-01 01:38:17,梧桐树扎根A股“一箭双雕”,0.992092,1
5,2016-04-01 01:39:06,央行今年第一季度调查问卷显示：股票投资被挤出,0.909551,1
6,2016-04-01 01:54:09,“深港通”适时启动 最终如何影响两地市场？,0.746931,0
7,2016-04-01 02:18:11,今年深化经济体制改革重点确定 适时启动“深港,0.999867,1
8,2016-04-01 02:30:12,两融余额重回8800亿 时隔3个月再现月度净买入,0.546472,0
9,2016-04-01 02:32:12,深化资本市场改革 适时启动深港通,0.999866,1


#### 使用分位点

In [47]:
# quantile033 = combine_df['sentiment_value'].quantile([0.33]).iloc[0]
# quantile067 = combine_df['sentiment_value'].quantile([0.67]).iloc[0]

In [48]:
# quantile033, quantile067

(0.4224526875888923, 0.9030995071236522)

In [49]:
# sentiment_label_list = []
# for i in sentiment_list:
#     if i <= quantile033:
#         sentiment_label_list.append(-1)
#     elif quantile033 <= i <= quantile067:
#         sentiment_label_list.append(0)
#     else:
#         sentiment_label_list.append(1)

#### 使用具体值

In [97]:

sentiment_label_list = []
for i in sentiment_list:
    if i < 0.15:
        sentiment_label_list.append(-1)
    elif 0.15 <= i <= 0.8:
        sentiment_label_list.append(0)
    else:
        sentiment_label_list.append(1)

In [98]:
sentiment_label_list[0:10]

[1, 0, 0, 1, 1, 1, 0, 1, 0, 1]

In [99]:
sentiment_list[0:10]

[0.8971616704178463,
 0.2504854889989895,
 0.3046549302317092,
 0.991793438939429,
 0.9920923196743372,
 0.9095511870707171,
 0.7469307418918795,
 0.9998666838277758,
 0.5464716060533193,
 0.9998660304154927]

In [100]:
new_series = pd.Series(sentiment_label_list)
new_series.describe()

count    112380.000000
mean          0.265786
std           0.730009
min          -1.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
dtype: float64

In [101]:
combine_df['sentiment_label'] = sentiment_label_list

In [102]:
combine_df.head()

,date,news_title,sentiment_value,sentiment_label
0,2016-04-01 00:03:56,等待非农就业数据 美股午盘维持小幅上扬,0.897162,1
1,2016-04-01 01:01:06,4月1日股市早盘内参：上交所决定4月8日起暂停*S,0.250485,0
2,2016-04-01 01:19:11,净买入再超20亿元 沪股通扫货四大券商股,0.304655,0
3,2016-04-01 01:22:06,专家:三月反攻圆满收官 四月行情值得期待,0.991793,1
4,2016-04-01 01:38:17,梧桐树扎根A股“一箭双雕”,0.992092,1


In [103]:
# 暂存一份
combine_df.to_csv('combine_df_with_sentimetnLabel.csv', index=False)

In [119]:
combine_df = pd.read_csv('combine_df_with_sentimetnLabel.csv',header = 0)

In [120]:
# combine_df.rename(columns={'发表时间':'date', '评论标题':'news_title'}, inplace=True)

In [121]:
combine_df.head()

,date,news_title,sentiment_value,sentiment_label
0,2016-04-01 00:03:56,等待非农就业数据 美股午盘维持小幅上扬,0.897162,1
1,2016-04-01 01:01:06,4月1日股市早盘内参：上交所决定4月8日起暂停*S,0.250485,0
2,2016-04-01 01:19:11,净买入再超20亿元 沪股通扫货四大券商股,0.304655,0
3,2016-04-01 01:22:06,专家:三月反攻圆满收官 四月行情值得期待,0.991793,1
4,2016-04-01 01:38:17,梧桐树扎根A股“一箭双雕”,0.992092,1


In [122]:
#将label 按照日期统计和
new_time = pd.to_datetime(combine_df['date']).apply(lambda x:x.strftime('%Y%m%d'))

In [125]:
new_time.shape, combine_df.shape

((112379,), (112379, 4))

In [126]:
combine_df['date'] = new_time

In [128]:
combine_df.tail()

,date,news_title,sentiment_value,sentiment_label
112374,20170930,国庆出境自驾游市场升温 年轻群体占比上升,0.584701,0
112375,20170930,新华社解读央行对普惠金融实施定向降准：结构性,0.996771,1
112376,20170930,青岛出台楼市调控政策 严格控制住宅用地价格,0.132093,-1
112377,20170930,一文读懂央行定向降准：6大疑问逐个解析(深度),0.977962,1
112378,20170930,海南实行“多证合一、一照一码”,0.450917,0


In [134]:
combine_df.groupby(['date', 'sentiment_label']).size().head(20)

date      sentiment_label
20160401  -1                 11
           0                 27
           1                 35
20160402  -1                  5
           0                 17
           1                 30
20160403  -1                  7
           0                 23
           1                 15
20160404  -1                  5
           0                 13
           1                 22
20160405  -1                 14
           0                 24
           1                 31
20160406  -1                 12
           0                 33
           1                 31
20160407  -1                 13
           0                 27
dtype: int64

In [135]:
date_sentiment_value_df = combine_df.groupby(['date', ])['sentiment_label'].sum()

In [136]:
date_sentiment_value_df.head(10)

date
20160401    24
20160402    25
20160403     8
20160404    17
20160405    17
20160406    19
20160407    15
20160408    31
20160409    25
20160410     9
Name: sentiment_label, dtype: int64

In [138]:
date_sentiment_value_df.shape

(548,)

In [141]:
date_sentiment_value_df = date_sentiment_value_df.reset_index()

In [147]:
date_sentiment_value_df.head()

,trade_date,sentiment_label
0,20160401,24
1,20160402,25
2,20160403,8
3,20160404,17
4,20160405,17


In [145]:
date_sentiment_value_df.rename(columns={'date':'trade_date', }, inplace=True)

In [146]:
date_sentiment_value_df.to_csv('sentiment_final_count_withdate.csv', index=False)